In [1]:
import tensorflow as tf
import os
import keras.backend as K
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import l2

from keras.preprocessing.image import *

import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm
from glob import glob
import multiprocessing

from sklearn.model_selection import train_test_split
from collections import Counter
from keras import backend as K
from keras.utils import multi_gpu_model
from dataset import *

task_names = ['skirt_length', 'coat_length', 'collar_design', 'neck_design', 'neckline_design', 'pant_length', 'sleeve_length', 'lapel_design']



Using TensorFlow backend.


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))

In [3]:
task_name = task_names[5]
df = pd.read_csv('data/Annotations/label.csv', header=None)
df.columns = ['filename', 'label_name', 'label']

df.label_name = df.label_name.str.replace('_labels', '')
df = df[df.label_name == task_name]
df = df.sample(frac=1).reset_index(drop=True) # shuffle

c = Counter(df.label_name)
label_count = dict([(x, len(df[df.label_name == x].label.values[0])) for x in c.keys()])
label_names = list(label_count.keys())
n = len(df)
fnames = df['filename'].values
width = 399

y = [np.zeros((n, label_count[x])) for x in label_count.keys()]
df = df.reset_index(drop=True)

def f(index):
    if os.path.isfile('data/'+fnames[index]):
        return index, cv2.resize(cv2.imread('data/'+fnames[index]), (width, width))

for i in range(n):
    label_name = df.label_name[i]
    label = df.label[i]
    y[label_names.index(label_name)][i, label.find('y')] = 1
    #y[label_names.index(label_name)][i, label.find('m')] = 0.5

X = np.zeros((n, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img[:,:,::-1]

n_train = int(n*0.8)

X_train = X[:n_train]
X_valid = X[n_train:]
y_train = [x[:n_train] for x in y]
y_valid = [x[n_train:] for x in y]

gen_train = Generator(X_train, y_train, batch_size=16, aug=True)

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
base_model = InceptionV3(weights='imagenet', input_shape=(width, width, 3), include_top=False, pooling = 'avg')

input_tensor = Input((width, width, 3))
x = input_tensor
x = Lambda(preprocess_input)(x)
x = base_model(x)
x = Dropout(0.5)(x)
x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

model = Model(input_tensor, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

gpumodel = multi_gpu_model(model, n_gpus)



100%|██████████| 7460/7460 [00:18<00:00, 410.51it/s]


In [4]:
gpumodel.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save_weights('model_%s.h5' % model_name)

Epoch 1/3
374/374 [==============================] - 337s 900ms/step - loss: 0.9957 - acc: 0.6277 - val_loss: 0.5782 - val_acc: 0.7929
Epoch 2/3
374/374 [==============================] - 252s 674ms/step - loss: 0.5734 - acc: 0.8021 - val_loss: 0.4196 - val_acc: 0.8425
Epoch 3/3
374/374 [==============================] - 251s 672ms/step - loss: 0.4287 - acc: 0.8533 - val_loss: 0.4266 - val_acc: 0.8485


In [5]:
gpumodel.compile(optimizer=Adam(0.000025), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=2, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save_weights('model_%s.h5' % model_name)

Epoch 1/2
374/374 [==============================] - 334s 893ms/step - loss: 0.2545 - acc: 0.9121 - val_loss: 0.3737 - val_acc: 0.8666
Epoch 2/2
374/374 [==============================] - 255s 681ms/step - loss: 0.1838 - acc: 0.9403 - val_loss: 0.3774 - val_acc: 0.8592


In [6]:
gpumodel.compile(optimizer=Adam(0.00000625), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save_weights('model_%s.h5' % model_name)

Epoch 1/3
374/374 [==============================] - 348s 929ms/step - loss: 0.1284 - acc: 0.9596 - val_loss: 0.3801 - val_acc: 0.8700
Epoch 2/3
374/374 [==============================] - 256s 685ms/step - loss: 0.1096 - acc: 0.9677 - val_loss: 0.3851 - val_acc: 0.8693
Epoch 3/3
374/374 [==============================] - 256s 685ms/step - loss: 0.1059 - acc: 0.9696 - val_loss: 0.3942 - val_acc: 0.8693


In [7]:
gpumodel.compile(optimizer=Adam(0.00000425), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save_weights('model_%s.h5' % model_name)

Epoch 1/1
374/374 [==============================] - 351s 939ms/step - loss: 0.0934 - acc: 0.9719 - val_loss: 0.3947 - val_acc: 0.8653


In [8]:
gpumodel.compile(optimizer=SGD(lr=1e-8, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save_weights('model_%s.h5' % model_name)

Epoch 1/1
374/374 [==============================] - 321s 858ms/step - loss: 0.0851 - acc: 0.9758 - val_loss: 0.3933 - val_acc: 0.8646
